In [8]:
import numpy as np
import pandas as pd
import math
import synthetic_generate as sg
np.set_printoptions(precision=2)


def main():
    data_type="GAUSSIAN" 
    d = 8
    est_type="sw"
    repeat=50
    repeat_est=repeat
    ratio= 0.9
    N=10000
    low,high = -3, 3
    # set index of bins
    beta = 1
    bin_width = beta/d
    bin_idxs = np.linspace(bin_width/2, beta-bin_width/2, d)

    # generate data
    data = sg.generate_synthetic_data(data_type=data_type,
            low=low,high=high,n=N,beta=beta)
    histo_true,_ = np.histogram(a=data, range=(0,beta), bins=d)
    q_true = np.reshape(histo_true/np.sum(histo_true),(1,d))
    print("synthetic data generated, N=%d." % (data.size))

    # set epsilon
    eps_grid = np.linspace(0.5,2.5,5)

    result = np.zeros((eps_grid.size, 4))
    for i in range(len(eps_grid)):

        print('\n==== eps=%.2f, repeat=%d ====' %(eps_grid[i],repeat))
        
        _,_,q_est = sg.DP_dist_estimation(histo_true=histo_true,bin_idxs=bin_idxs,
            est_type=est_type,eps=eps_grid[i]*ratio, repeat=repeat_est,q_est=q_true)
        print('initial estimation complete, eps=%.2f' 
              %(eps_grid[i]*ratio))

        result[i,0],result[i,1],_ = sg.DP_dist_estimation(
            histo_true=histo_true, bin_idxs=bin_idxs,
            est_type='aaa', eps=eps_grid[i]*(1-ratio),
            repeat=repeat, q_est = q_est)
        print('aaa estimation complete, eps= %.2f' 
              %(eps_grid[i]*(1-ratio)))
        
        result[i,2],result[i,3],_ = sg.DP_dist_estimation(
            histo_true=histo_true, bin_idxs=bin_idxs,
            est_type=est_type, eps=eps_grid[i],
            repeat=repeat, q_est=q_true)
        print('%s estimation complete, eps=%.2f' 
              %(est_type,eps_grid[i]))
    
    filename = 'data/result_%s_%s_%d(%.1f,%d).csv' % (data_type,
                        est_type, d, ratio, repeat)
    temp = {'eps_grid': eps_grid,
        'mse_aaa': result[:, 0],
        'wass_aaa': result[:, 1],
        'mse_est': result[:, 2],
        'wass_est': result[:, 3]}
    pd.DataFrame(temp).to_csv(filename)
    print('task complete!')
    return temp

if __name__=='__main__':
    result = main()


synthetic data generated, N=10000.

==== eps=0.50, repeat=50 ====
initial estimation complete, eps=0.45
aaa estimation complete, eps= 0.05


c:\Users\feiwei\GitHub\OptimalFreqEstimatePy\a3mV2.py:48: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq = A_eq, b_eq = b_eq, bounds=bounds)


sw estimation complete, eps=0.50

==== eps=1.00, repeat=50 ====
initial estimation complete, eps=0.90
aaa estimation complete, eps= 0.10
sw estimation complete, eps=1.00

==== eps=1.50, repeat=50 ====
initial estimation complete, eps=1.35
aaa estimation complete, eps= 0.15
sw estimation complete, eps=1.50

==== eps=2.00, repeat=50 ====
initial estimation complete, eps=1.80
aaa estimation complete, eps= 0.20
sw estimation complete, eps=2.00

==== eps=2.50, repeat=50 ====
initial estimation complete, eps=2.25
aaa estimation complete, eps= 0.25
sw estimation complete, eps=2.50
task complete!
